In [ ]:
pip install python-dotenv spotipy

In [ ]:
pip install --upgrade pip

In [ ]:
pip install tkinter

In [ ]:
pip install scikit-image

In [ ]:
pip install scikit-learn

In [ ]:
pip install matplotlib

In [ ]:
pip install pandas

In [10]:
import csv
import os
import re

import spotipy
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyClientCredentials

from skimage import io
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import pandas as pd
from PIL import Image, ImageTk


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

auth_manager1 = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager1)

In [11]:
def data_cleaning_and_conversion():
    spotify_data_1 = pd.read_csv('dataSet-Final-1.csv')
    spotify_data_1.duplicated(subset=['track_name']).sum()
    spotify_data = spotify_data_1.drop_duplicates(subset=['track_name'])
    spotify_data.duplicated(subset=['track_name']).sum()
    spotify_features_df = spotify_data
    release_year_OHE = pd.get_dummies(spotify_features_df.release_year)

    scaled_features = MinMaxScaler().fit_transform([
    spotify_features_df['acousticness'].values,
    spotify_features_df['danceability'].values,
    spotify_features_df['energy'].values,
    spotify_features_df['instrumentalness'].values,
    spotify_features_df['liveness'].values,
    spotify_features_df['loudness'].values,
    spotify_features_df['speechiness'].values,
    spotify_features_df['tempo'].values,
    spotify_features_df['valence'].values,
    ])

    spotify_features_df[['acousticness','danceability','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence']] = scaled_features.T
    spotify_features_df = spotify_features_df.drop('artist', axis = 1)
    spotify_features_df = spotify_features_df.drop('track_name', axis = 1)
    spotify_features_df = spotify_features_df.drop('popularity',axis = 1)
    spotify_features_df = spotify_features_df.drop('time_signature', axis = 1)
    spotify_features_df = spotify_features_df.drop('release_year',axis = 1)

    spotify_features_df = spotify_features_df.join(release_year_OHE)

    return spotify_features_df,spotify_data

In [12]:
def get_user_playlist(user_id):
    try:
        playlist_dic = {}
        playlist_cover_art = {}

        for i in sp.user_playlists(user_id)['items']:
            playlist_dic[i['name']] = i['uri'].split(':')[2]
            playlist_cover_art[i['uri'].split(':')[2]] = i['images'][0]['url']

        return playlist_dic
    
    except:
        return {}

In [13]:
def generate_playlist_df(playlist_name, playlist_dic, spotify_data):
    try:
        playlist = pd.DataFrame()

        for i, j in enumerate(sp.playlist(playlist_dic[playlist_name])['tracks']['items']):
            playlist.loc[i, 'artist'] = j['track']['artists'][0]['name']
            playlist.loc[i, 'track_name'] = j['track']['name']
            playlist.loc[i, 'track_id'] = j['track']['id']
            playlist.loc[i, 'url'] = j['track']['album']['images'][1]['url']
            playlist.loc[i, 'date_added'] = j['added_at']

        playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
        
        playlist = playlist[playlist['track_id'].isin(spotify_data['track_id'].values)].sort_values('date_added',ascending = False)

        return playlist 
    
    except:
        return playlist
        # return pd.Series([])

In [14]:
def visualize_cover_art(playlist_df):
    temp = playlist_df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))) , facecolor='#2192FF')
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(int(len(temp) / columns + 1), columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
        s='' 
        plt.xlabel(s.join(playlist_df['track_name'].values[i].split(' ')[:4]), fontsize = 10, fontweight='bold')
        plt.tight_layout(h_pad=0.8, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [15]:
def get_recommendation_images(playlist_df):
    temp = playlist_df['url'].values
    image_list = []
    track_names = []
    track_links = []
    pre_link = "https://open.spotify.com/track/"
    for i,url in enumerate(temp):
        # image = io.imread(url)
        # image_list.append(image)
        s=''
        track_names.append(s.join(playlist_df['track_name'].values[i].split(' ')[:4]))
        link = pre_link + playlist_df['track_id'].values[i]
        track_links.append(link)
    return track_names,track_links

In [16]:
# creating playlist vector

def generate_playlist_vector(spotify_features, playlist_df, weight_factor):
    
    spotify_features_playlist = spotify_features[spotify_features['track_id'].isin(playlist_df['track_id'].values)]
    spotify_features_playlist = spotify_features_playlist.merge(playlist_df[['track_id','date_added']], on = 'track_id', how = 'inner')
    
    spotify_features_nonplaylist = spotify_features[~spotify_features['track_id'].isin(playlist_df['track_id'].values)]
    
    playlist_feature_set = spotify_features_playlist.sort_values('date_added',ascending=False)
    
    
    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'days_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days)
        
    
    playlist_feature_set['weight'] = playlist_feature_set['days_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-3].mul(playlist_feature_set_weighted.weight.astype(int),0))   
    
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-3]
    

    
    return playlist_feature_set_weighted_final.sum(axis = 0), spotify_features_nonplaylist

In [17]:
def generate_recommendation(spotify_data, playlist_vector, nonplaylist_df):

    non_playlist = spotify_data[spotify_data['track_id'].isin(nonplaylist_df['track_id'].values)]
    non_playlist['sim'] = cosine_similarity(nonplaylist_df.drop(['track_id'], axis = 1).values, playlist_vector.drop(labels = 'track_id').values.reshape(1, -1))[:, 0]
    non_playlist_top15 = non_playlist.sort_values('sim',ascending = False).head(15)
    non_playlist_top15['url'] = non_playlist_top15['track_id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return  non_playlist_top15

In [31]:
import tkinter as tk
from tkinter import ttk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
import webbrowser
from PIL import Image, ImageTk


class RootApp(tk.Tk):

    def __init__(self):
        super().__init__()

        self.geometry("640x480")
        self.title("SPOTIFY SONG RECOMMENDATION ENGINE")
        self.configure(bg="lightgreen")

        self._frame = None
        self.switch_frame(Tab1)

    def switch_frame(self, frame_class):

        new_frame = frame_class(self)

        if self._frame:
            self._frame.destroy()

        self._frame = new_frame
        self._frame.pack()

class Tab1(tk.Frame):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.profile_link = '' # default value at start

        self._frame = None
        self.switch_frame(Tab1_Frame1)

    def switch_frame(self, frame_class):

        new_frame = frame_class(self)

        if self._frame:
            self._frame.destroy()

        self._frame = new_frame
        self._frame.pack()


class Tab1_Frame1(tk.Frame):

    def __init__(self, *args, **kwargs):
        global profile_link

        super().__init__(*args, **kwargs)
        self.configure(bg="lightgreen")

        img = Image.open("bg.png")
        resized = img.resize((300,200))
        Tab1_Frame1.img_bg = ImageTk.PhotoImage(resized)
        self.image_bg = tk.Label(self, image = Tab1_Frame1.img_bg)
        self.image_bg.image = Tab1_Frame1.img_bg
        self.image_bg.pack(pady=80)

        self.label = tk.Label(self, text="Enter your Spotify Profile Link : ",bg = "lightgreen",font=("Helvatical bold",30))
        self.label.pack(pady= 10)

        self.newWindow = None

        self.link_entry = tk.Entry(self, width= 100, bg = "white",fg= "black", font=("Helvatical bold",15))
        self.link_entry.place(x=0, y=430, height=35)

        profile_link = self.link_entry.get()

        self.button = tk.Button(self, text="Submit", bg="green",fg="white",font=("Helvatical bold",13),command=self.switch_frame)
        self.button.pack(pady=40)

    def switch_frame(self):
        value = self.link_entry.get()
        self.master.profile_link = value
        self.master.switch_frame(Tab1_Frame2)

    def new_window(self):

        if self.newWindow:
            self.newWindow.destroy()

        self.newWindow = tk.Frame(self)

        self.profile_link = self.link_entry.get()
        self.app = logwindow(self.newWindow, self.profile_link)     


class Tab1_Frame2(tk.Frame):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.configure(bg="lightgreen")

        self.profile_link = self.master.profile_link
        self.user_profile_link = list(str(self.profile_link).split("/"))
        self.user_id = self.user_profile_link[4]
        self.user_id = self.user_id[:self.user_profile_link[4].index('?')]
        print(self.user_id)

        img = Image.open("bg.png")
        resized = img.resize((300,200))
        Tab1_Frame2.img_bg = ImageTk.PhotoImage(resized)
        self.image_bg = tk.Label(self, image = Tab1_Frame2.img_bg)
        self.image_bg.image = Tab1_Frame2.img_bg
        self.image_bg.pack(pady=80)

        self.button = tk.Button(self, text="Back", font=("Helvatical bold",13),bg = "green",fg = "white" ,command=lambda: self.master.switch_frame(Tab1_Frame1))
        self.button.place(x = 10, y = 300)

        spotify_features_df,spotify_data = data_cleaning_and_conversion()
        playlist_dic = get_user_playlist(self.user_id)

        if(playlist_dic == {}):
            tk.messagebox.showinfo("Error","Error in the link provided...\n Please provide valid link !")
            return

        self.clicked = tk.StringVar()
        self.clicked.set("Select a Playlist")

        def show():
            playlist_name = self.clicked.get()
            print(playlist_name)
            playlist_df = generate_playlist_df(playlist_name, playlist_dic, spotify_data)
            if(playlist_df.empty):
                tk.messagebox.showinfo("Error","Playlist may contains missing data...\n Retry !!!")
                return
            playlist_vector, nonplaylist_df = generate_playlist_vector(spotify_features_df, playlist_df, 1.2)
            top15 = generate_recommendation(spotify_data, playlist_vector, nonplaylist_df)  
            track_names,track_links = get_recommendation_images(top15)

            track_names = track_names[:11]
            
            track_links = track_links[:11]
            
            def callback(event, url):
                webbrowser.open_new(url)
            
            for track,link in zip(track_names,track_links):
                a = tk.StringVar()
                a = track
                trackLabel = tk.Label(self, text= a,font=("Helvatical bold",12), bg= "lightgreen")
                trackLabel.bind("<Button-1>",lambda e, x = link: callback(e, x))
                trackLabel.pack(side= tk.BOTTOM, pady = 7)
                

        options = list(playlist_dic.keys())

        drop = tk.OptionMenu( self , self.clicked , *options )
        drop.config(fg="black",font=("Helvatical bold",13))
        drop["menu"].config(bg = "lightgreen", font = ("Helvatical bold",13))
        drop.place(x=80,y=300)

        self.button = tk.Button( self , text = "Get Recommendation" , bg="green", fg="white",font=("Helvatical bold",14),command = show )
        self.button.pack(pady = 5)

    def new_window(self):
        self.newWindow = tk.Toplevel(self.master)
        self.profile_link = self.link_entry.get()


if __name__ == "__main__":
    root = RootApp()
    root.mainloop()

0herhw6ubfo9aigw4uf02knjn


C:\Users\gkabi\AppData\Local\Temp\ipykernel_10340\1551521546.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spotify_features_df[['acousticness','danceability','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence']] = scaled_features.T


Slow poison 🙂🤍


C:\Users\gkabi\AppData\Local\Temp\ipykernel_10340\1131795710.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist['sim'] = cosine_similarity(nonplaylist_df.drop(['track_id'], axis = 1).values, playlist_vector.drop(labels = 'track_id').values.reshape(1, -1))[:, 0]
